In [1]:
from facebook_utils import start_browser, sign_in, get_links, get_info, get_friends, validate_profile
import os
import sys
import json
from glob import glob 
import pandas as pd
import time
from tqdm.notebook import tqdm

with open('C:/Users/vishw/OneDrive/Desktop/Projects/DataSec_OSINT/UI/cred.json') as f:
    cred = json.load(f)
    username = cred['username']
    password = cred['password']


In [10]:
names = pd.read_excel('Dataset/Accuracy Profiles.xlsx')
names = names.dropna()

In [11]:
def get_id_from_url(url):
    name = url.split('/')[3].split('?')[0]
    if name == 'profile.php':
        name = url.split('/')[3].split('?')[1].split('&')[0]

    return name


names['id'] = names['URLs'].apply(get_id_from_url)
## Remove if id in search or photo
names = names[~names['id'].str.contains('search')]
names = names[~names['id'].str.contains('photo')]


In [12]:
browser = start_browser()
sign_in(browser, username, password)

Opening Browser...
Logging in vishwa20@outlook.com automatically...


In [14]:
recognitions = []
for name,link,id in tqdm(names.values[40:]):
    try:
        target_folder = 'C:/Users/vishw/OneDrive/Desktop/Projects/DataSec_OSINT/Utils/Dataset/Images/'+name+'/'
        print(glob(target_folder+"*"))
        assert len(glob(target_folder+"*")) > 0
        ## Calculate the time taken to validate the profile
        st = time.time()
        found_link, all_links = validate_profile(name, glob(target_folder+"*"), browser)
        et = time.time()
        recognitions.append([name, link, id, found_link, get_id_from_url(found_link), [get_id_from_url(i) for i in all_links], et-st]) 
        pd.DataFrame(recognitions, columns = ['Name', 'Link', 'ID', 'Found_Link', 'Found_ID', 'Searched_IDs','Time']).to_excel('recognitions.xlsx', index = False)
    except Exception as e:
        print(e)
        recognitions.append([name, link, id, None, None, None])

## Save the recognitions list as a dataframe


  0%|          | 0/8 [00:00<?, ?it/s]

['C:/Users/vishw/OneDrive/Desktop/Projects/DataSec_OSINT/Utils/Dataset/Images/Jordi Pahul Robredo\\Jordi Pahul Robredo 1.jpg', 'C:/Users/vishw/OneDrive/Desktop/Projects/DataSec_OSINT/Utils/Dataset/Images/Jordi Pahul Robredo\\Jordi Pahul Robredo 2.jpg']
['https://www.facebook.com/jordi.robredo']
Fetching profile  1
Recognising and counting targets.
['C:/Users/vishw/OneDrive/Desktop/Projects/DataSec_OSINT/Utils/Dataset/Images/Xavi Moreno\\Xavi Moreno 1.jpg', 'C:/Users/vishw/OneDrive/Desktop/Projects/DataSec_OSINT/Utils/Dataset/Images/Xavi Moreno\\Xavi Moreno 2.jpg', 'C:/Users/vishw/OneDrive/Desktop/Projects/DataSec_OSINT/Utils/Dataset/Images/Xavi Moreno\\Xavi Moreno 3.jpg']
['https://www.facebook.com/xavimoreno', 'https://www.facebook.com/profile.php?id=100009004917171', 'https://www.facebook.com/javier.morenogutierrez.9', 'https://www.facebook.com/xavi.moreno.144', 'https://www.facebook.com/profile.php?id=61554434891707']
Fetching profile  1
Fetching profile  2
Fetching profile  3
Fetch

In [15]:
results = pd.concat([pd.read_excel('recognitions_1.xlsx').dropna(), pd.read_excel('recognitions.xlsx').dropna()])

In [16]:
(results['ID'] == results['Found_ID']).sum()/results.shape[0]

0.5675675675675675

In [22]:
## Check how many IDs are found in the searched IDs
results['ID_in_searched'] = results.apply(lambda x: x['ID'] in x['Searched_IDs'], axis = 1)
results['Found_ID_in_searched'] = results.apply(lambda x: x['Found_ID'] in x['Searched_IDs'], axis = 1)

results['ID_in_searched'].sum()/results.shape[0]

0.7297297297297297

In [26]:
(results[results['ID_in_searched']]['ID'] == results[results['ID_in_searched']]['Found_ID']).sum()/results[results['ID_in_searched']].shape[0]

0.7777777777777778

In [29]:
results['Time'].mean()

41.51121940483918

In [30]:
results

,Name,Link,ID,Found_Link,Found_ID,Searched_IDs,Time,ID_in_searched,Found_ID_in_searched
0,Gianluigi Scanu,https://www.facebook.com/furi.soru?locale=es_LA,furi.soru,https://www.facebook.com/furi.soru,furi.soru,"['gianluigi.scanu.98', 'gianluigi.scanu.9', 'f...",38.781161,True,True
1,Leonardo Williams,"""https://www.facebook.com/profile.php?id=10009...",id=100090379008590,https://www.facebook.com/profile.php?id=100089...,id=100089882193313,"['id=100089882193313', 'id=100008314452474', '...",35.935353,False,True
2,Giorgio Cerro,"""https://www.facebook.com/giorgio.cerro?locale...",giorgio.cerro,https://www.facebook.com/giorgio.cerro,giorgio.cerro,"['giorgio.cerro', 'giorgio.cerro.9', 'giorgio....",43.099581,True,True
3,Daniele Greco,"""https://www.facebook.com/daniele.greco.54?loc...",daniele.greco.54,https://www.facebook.com/daniele.greco.148116,daniele.greco.148116,"['daniele.greco.148116', 'id=100088522624763',...",42.250345,False,True
4,Emanuele Piras,"""https://www.facebook.com/emanuele.piras.75?lo...",emanuele.piras.75,https://www.facebook.com/emanuele.piras.75,emanuele.piras.75,"['emanuele.piras.1042', 'id=100008927563737', ...",42.228680,True,True
5,Alessio Mei,"""https://www.facebook.com/alessio.mei.52?local...",alessio.mei.52,https://www.facebook.com/alessio.mei.54,alessio.mei.54,"['alessio.mei.52', 'alee.mei', 'alessio.mei.75...",43.029809,True,True
6,Giuseppe Pantano,"""https://www.facebook.com/OfficialNuraT?locale...",OfficialNuraT,https://www.facebook.com/OfficialNuraT,OfficialNuraT,"['OfficialNuraT', 'giuseppe.pantano.12914', 'u...",44.124747,True,True
7,Fabio Attanasio,"""https://www.facebook.com/fa.attanasio?locale=...",fa.attanasio,https://www.facebook.com/fa.attanasio,fa.attanasio,"['fa.bi.1654700', 'fa.attanasio', 'id=10001043...",39.285325,True,True
8,Gianluca Saderi,"""https://www.facebook.com/giangi.sade?locale=e...",giangi.sade,https://www.facebook.com/giangi.sade,giangi.sade,"['giangi.sade', 'gianlusamy', 'giuliana.bedini...",42.419747,True,True
9,Domenico Sirico,"""https://www.facebook.com/dsirico?locale=es_LA",dsirico,https://www.facebook.com/dsirico,dsirico,"['domenico.sirico.94', 'beach03', 'dsirico', '...",39.420003,True,True
